"https://ssl.pstatic.net/imgstock/upload/research/debenture/1595205128882.pdf"

In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 섹션 분리\n",
    "- 2, 3만 사용"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "metadata": {},
   "outputs": [],
   "source": [
    "def tidy_sentences(section):\n",
    "    sentence_enders = re.compile(r'((?<=[함음됨임봄짐움])(\\s*\\n|\\.|;)|(?<=다)\\.)\\s*')\n",
    "    splits = list((m.start(), m.end()) for m in re.finditer(sentence_enders, section))\n",
    "    starts = [0] + [i[1] for i in splits]\n",
    "    ends = [i[0] for i in splits]\n",
    "    sentences = [section[start:end] for start, end in zip(starts[:-1], ends)]\n",
    "    for i, s in enumerate(sentences):\n",
    "        sentences[i] = (s.replace('\\n', ' ').replace(' ', ' ')) + '.'\n",
    "\n",
    "    text = '\\n'.join(sentences) if len(sentences) > 0 else ''\n",
    "    return sentences, text"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "metadata": {},
   "outputs": [],
   "source": [
    "import re \n",
    "\n",
    "def preprocess_minutes(minutes):\n",
    "\n",
    "    pos = re.search('(.?국내외\\s?경제\\s?동향.?과 관련하여,?|\\(가\\).+경제전망.*|\\(가\\) 국내외 경제동향 및 평가)\\n?\\s*일부 위원은', minutes, re.MULTILINE)\n",
    "    s1 = pos.start() if pos else -1\n",
    "    pos = re.search('(.?외환.?국제금융\\s?동향.?과 관련하여.*|\\(나\\) 외환.국제금융\\s?(및 금융시장)?\\s?동향)\\n?\\s*(일부 위원은|대부분의 위원들은)', minutes,re.MULTILINE)\n",
    "    s2 = pos.start() if pos else -1\n",
    "    pos = re.search('(.?금융시장\\s?동향.?과 관련하여,?|\\(다\\) 금융시장\\s?동향)\\n?\\s*일부 위원은', minutes, re.MULTILINE)\n",
    "    s3 = pos.start() if pos else -1\n",
    "    pos = re.search('((\\((다|라)\\) )?.?통화정책\\s?방향.?에 관한 토론,?|이상과 같은 의견\\s?교환을 바탕으로.*통화정책\\s?방향.*에.*토론.*)\\n?', minutes,re.MULTILINE)\n",
    "    s4 = pos.start() if pos else -1\n",
    "    pos = re.search('(\\(4\\) 정부측 열석자 발언.*)\\n?', minutes, re.MULTILINE)\n",
    "    s5 = pos.start() if pos else -1\n",
    "    pos = re.search('(\\(.*\\) 한국은행 기준금리 결정에 관한 위원별 의견\\s?개진|이상과 같은 토론에 이어 .* 관한 위원별 의견개진이 있었음.*)\\n?', minutes,re.MULTILINE)\n",
    "    s6 = pos.start() if pos else -1\n",
    "    positer = re.finditer('(\\(\\s?.*\\s?\\) ()(심의결과|토의결론))\\n?', minutes, re.MULTILINE)\n",
    "    s7 = [pos.start() for pos in positer if pos.start() > s6]\n",
    "    s7 = s7[0] if s7 else -1\n",
    "\n",
    "    # 국내외 경제동향\n",
    "    bos = s1\n",
    "    eos = s2\n",
    "    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''\n",
    "    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)\n",
    "    bos = pos.start() if pos else -1\n",
    "    section = section[bos:] if bos >= 0 else section\n",
    "    section1, section1_txt = tidy_sentences(section)\n",
    "\n",
    "    # 외환․국제금융 동향\n",
    "    bos = s2\n",
    "    eos = s3 if s3 >= 0 else s4\n",
    "    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''\n",
    "    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)\n",
    "    bos = pos.start() if pos else -1\n",
    "    section = section[bos:] if bos >= 0 else section\n",
    "    section2, section2_txt = tidy_sentences(section)\n",
    "    #print(section)\n",
    "\n",
    "    # 금융시장 동향\n",
    "    bos = s3\n",
    "    eos = s4\n",
    "    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''\n",
    "    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)\n",
    "    bos = pos.start() if pos else -1\n",
    "    section = section[bos:] if bos >= 0 else section\n",
    "    section3, section3_txt = tidy_sentences(section)\n",
    "\n",
    "    # 통화정책방향\n",
    "    bos = s4\n",
    "    eos = s5 if s5 >= 0 else s6 if s6 >= 0 else s7\n",
    "    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''\n",
    "    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)\n",
    "    bos = pos.start() if pos else -1\n",
    "    section = section[bos:] if bos >= 0 else section\n",
    "    section4, section4_txt = tidy_sentences(section)\n",
    "\n",
    "    # 위원별 의견 개진\n",
    "    bos = s6\n",
    "    eos = s7\n",
    "    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''\n",
    "    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)\n",
    "    bos = pos.start() if pos else -1\n",
    "    section = section[bos:] if bos >= 0 else section\n",
    "    section5, section5_txt = tidy_sentences(section)\n",
    "\n",
    "    # 정부측 열석자 발언\n",
    "    bos = s5\n",
    "    eos = s6\n",
    "    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''\n",
    "    pos = re.search('정부측 열석자 발언', section, re.MULTILINE)\n",
    "    bos = pos.end() + 1 if pos else -1\n",
    "    section = section[bos:] if bos >= 0 else section\n",
    "    section6, section6_txt = tidy_sentences(section)\n",
    "\n",
    "    sections = ['Economic Situation', 'Foreign Currency', 'Financial Markets',\n",
    "                'Monetary Policy', 'Participants’ Views', 'Government’s View']\n",
    "    section_texts = (section1, section2, section3, section4, section5, section6)\n",
    "\n",
    "\n",
    "    return sections, section_texts"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.7.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}

ModuleNotFoundError: No module named 'pdf2text'